# Первичный анализ данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np

In [ ]:
import cv2
from tqdm import tqdm
import torch
import torchvision
random_seed = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

## Загрузка данных

In [ ]:
# загружаем обучающую выборку 
train_data = torchvision.datasets.MNIST(
    "mnist_content", train=True, transform=None, download=False
)

# разделяем обучающую выборку на обучающую и валидационную выборки
# 70% для обучения, 30% для валидации
train_size = int(len(train_data) * 0.7)
valid_size = len(train_data) - train_size
train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=None, download=True
)

In [ ]:
# генерируем имена колонок для пикселей изображения
img_col_names = [f"{i}x{j}" for i in range(1, 29) for j in range(1, 29)]

In [ ]:
train_df = []
stream = tqdm(train_data, desc="Train Data Processing")
for img, label in stream:
    np_img = np.array(img)
    train_df.append([label] + np_img.flatten().tolist())

train_df = pd.DataFrame(train_df, columns=["label"] + img_col_names)
train_df.shape

In [ ]:
valid_df = []
stream = tqdm(valid_data, desc="Valid Data Processing")
for img, label in stream:
    np_img = np.array(img)
    valid_df.append([label] + np_img.flatten().tolist())

valid_df = pd.DataFrame(valid_df, columns=["label"] + img_col_names)
valid_df.shape

In [ ]:
test_df = []
stream = tqdm(test_data, desc="Test Data Processing")
for img, label in stream:
    np_img = np.array(img)
    test_df.append([label] + np_img.flatten().tolist())

test_df = pd.DataFrame(test_df, columns=["label"] + img_col_names)
test_df.shape

## Обучающая выборка

In [ ]:
# Общая информация о данных: размер датасета, тип данных
train_df.info()

In [ ]:
# Посмотрим на то как представленны данные 
train_df.head()

In [ ]:
#Кол-во уникальных значений в первом столбце 
sorted(train_df['label'].unique())

mnist_train.csv содержит 60,000 строк, 785 колонок. Первая колонка- лэйбл картинки, содержит значения от 0-9. Колонки от 1-785 (1х1 - 28х28) сосдержат значения каждого пиксля изображения. Т.е в датасете представлено 60,000 примеров изображения для обучения 

Рассмотрим информацию о данных для каждой колонки отдельно для того чтобы оценить есть ли пропуски в данных 

In [ ]:
train_df.isnull().sum()

In [ ]:
for i in range(1, 785):
    train_df[train_df.columns.tolist()[i]].info()

Пропуски в данных отсутсвуют 

Рассмотрим соотношение значений в целевой колонке - колонке лэйблов изображений
Представим информацию в числовом виде и в виде графика 

In [ ]:
train_df['label'].value_counts()

In [ ]:
# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше
# TODO: добавить сверху каждого бина число = кол-во лейблов данного класса 
    # это можно сделать через _fig.bar_label(_fig.containers[0])
plt.hist(train_df['label'],rwidth=0.9)
plt.show()

Дисбаланса в данных нет, датасет содержит примерно равное количество примеров изображений по каждой цифре

Рассмотрим графики соотношения значений для остальных колонок

In [ ]:
# Мне кажется, что это не очень понятный график
# может быть просто посмотреть среднее значение, медиану, моду, без графиков, а фисто значения?
@interact 
def choose_pixel(i=widgets.IntSlider(value=1, min=1, max=784)): 
    plt.hist(data=train_df, x=train_df.columns.tolist()[i],bins=25,align='mid')

Из графиков видно, что в колонках 1-785 преобладают значение 0 и 255

Обозначим среднее значение пикселя в изображении для разных цифр как средняя интенсивность.

In [ ]:
# Создадим датасет в который перенесем колонку с лэйблами и сопоставим каждому значению среднее значение 
df_labels=pd.DataFrame({'labels':train_df['label'],'intensity':train_df.iloc[:,1:].mean(axis=1)})

# TODO: добавить подписи к осям 
# TODO: добавить название графика 
# TODO: сделать картинку побольше
# TODO: добавить сверху каждого бина число = кол-во лейблов данного класса 
    # это можно сделать через _fig.bar_label(_fig.containers[0])
plt.bar(df_labels['labels'],df_labels['intensity'])
plt.show()


Из данного графика видно, что самая низкая интенсивность наблюдается у цифры 1 (что связано с особенностями ее написания), самой высокой интенсивностью обладают цифры 0 и 8. 

Графики распределения средней интенсивности по лэйблу

In [ ]:
# TODO: добавить подписи к осям 
# TODO: сделать картинку побольше

#fig, ax = plt.subplots(nrows=9,ncols=1,  figsize=[5, 5])
for i in range(10):
    plt.hist(df_labels['intensity'][df_labels['labels']==i],bins=85,range=(0,100))
    plt.title(f'Intensity Histohram for {i}')
    plt.show()

Полученные распределения близки к нормальному распределению, но некоторые графики имеют более высокую дисперсию, чем другие, так как написание цифр у разных людей отличается. Так например у 1 наименьший разброс, в связи с чем можно предположить, что написание 1 имеет меньше всего различий.

Посмотрим на выборочные изображения рукописных цифр 

In [ ]:
@ interact

def show_digits(num=widgets.IntSlider(value=0, min=0, max=9)):
    arr_of_pics=[]
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=[15, 10])
    for i in range (5):
        j=np.random.choice(train_df[train_df['label']==num].index)
        digit=np.array(train_df.loc[j])
        arr_of_pics.append(np.resize(digit,(28,28)))

    ax[0].imshow(arr_of_pics[0],cmap='gray')
    ax[1].imshow(arr_of_pics[1],cmap='gray')
    ax[2].imshow(arr_of_pics[2],cmap='gray')
    ax[3].imshow(arr_of_pics[3],cmap='gray')
    ax[4].imshow(arr_of_pics[4],cmap='gray')
    

    plt.show()